In [68]:
import random
from datetime import datetime, timedelta

# 随机生成虚拟数据的函数
def generate_game_data(num):
    game_data = []
    for i in range(num):
        release_days = random.randint(1, 3650)  # 随机生成发布天数
        release_date = (datetime.now() - timedelta(days=release_days)).strftime('%Y-%m-%d')
        
        iphone_download = random.randint(1000, 50000)
        iphone_revenue = round(random.uniform(10000, 5000000), 2)
        ipad_download = random.randint(100, 10000)
        ipad_revenue = round(random.uniform(1000, 500000), 2)
        total_download = iphone_download + ipad_download
        total_revenue = round(iphone_revenue + ipad_revenue, 2)
        dau = round(np.random.normal(loc=1000000, scale=500000), 2)  # 均值1000000，标准差500000

        game_data.append({
            'name': f'游戏名称 {i+1}',
            'release_date': release_date,
            'iphone_download': iphone_download,
            'iphone_revenue': iphone_revenue,
            'ipad_download': ipad_download,
            'ipad_revenue': ipad_revenue,
            'download': total_download,
            'revenue': total_revenue,
            'dau': max(1000, dau),
            'release_days': release_days,
        })
    return game_data

# 生成100条数据
game_data_list = generate_game_data(100)
# game_data_list[:1]

data = pd.DataFrame(game_data_list)
data.head(3)

,name,release_date,iphone_download,iphone_revenue,ipad_download,ipad_revenue,download,revenue,dau,release_days
0,游戏名称 1,2019-09-12,22726,1751910.32,2548,296169.80,25274,2048080.12,1021302.10,1930
1,游戏名称 2,2023-06-09,41277,2922510.98,1373,88570.17,42650,3011081.15,756849.39,564
2,游戏名称 3,2019-10-23,24205,2039016.99,165,188910.10,24370,2227927.09,1267071.50,1889


In [69]:
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             100 non-null    object 
 1   release_date     100 non-null    object 
 2   iphone_download  100 non-null    int64  
 3   iphone_revenue   100 non-null    float64
 4   ipad_download    100 non-null    int64  
 5   ipad_revenue     100 non-null    float64
 6   download         100 non-null    int64  
 7   revenue          100 non-null    float64
 8   dau              100 non-null    float64
 9   release_days     100 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 7.9+ KB


,name,release_date,iphone_download,iphone_revenue,ipad_download,ipad_revenue,download,revenue,dau,release_days
0,游戏名称 1,2019-09-12,22726,1751910.32,2548,296169.80,25274,2048080.12,1021302.10,1930
1,游戏名称 2,2023-06-09,41277,2922510.98,1373,88570.17,42650,3011081.15,756849.39,564
2,游戏名称 3,2019-10-23,24205,2039016.99,165,188910.10,24370,2227927.09,1267071.50,1889
3,游戏名称 4,2016-09-04,9593,4953411.32,6369,392072.36,15962,5345483.68,204984.93,3033
4,游戏名称 5,2023-11-05,3156,575957.45,191,15855.79,3347,591813.24,1085900.75,415


In [70]:
data.describe(include='all').T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
name,100,100,游戏名称 1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
release_date,100,99,2018-03-26,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
iphone_download,100.0,NaN,NaN,NaN,27761.78,12779.843451,2539.0,18449.5,27269.0,39228.5,49738.0
iphone_revenue,100.0,NaN,NaN,NaN,2413941.6059,1340287.394191,74151.67,1388150.4975,2457832.58,3232174.81,4977090.78
ipad_download,100.0,NaN,NaN,NaN,5219.7,2867.294838,126.0,2838.0,5573.0,7556.75,9903.0
ipad_revenue,100.0,NaN,NaN,NaN,240078.7087,151163.892043,7005.68,110777.7075,219657.465,392707.32,499781.98
download,100.0,NaN,NaN,NaN,32981.48,13310.282155,3347.0,24617.5,32357.0,43584.25,57544.0
revenue,100.0,NaN,NaN,NaN,2654020.3146,1360614.06603,245414.52,1624343.8425,2626347.075,3593655.5,5350838.21
dau,100.0,NaN,NaN,NaN,852818.0254,471397.616295,1000.0,553972.89,878913.325,1163452.785,2125382.9
release_days,100.0,NaN,NaN,NaN,1663.46,1063.270345,35.0,672.75,1544.0,2474.0,3612.0


In [71]:
import math
from pyecharts import options as opts
from pyecharts.charts import Scatter
from pyecharts.commons.utils import JsCode

from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

In [72]:
x = 'release_days'
y = 'revenue'
size = 'dau'
y_max = np.inf

x_data = data[x].tolist()

def cal_symbol_size(data: DataFrame, column: str, index: int, default_size: int = 1):
    s = data[column]
    rmin, rmax = s.min(), s.max()
    value = data.loc[index, column]
    _normalized = ((value - rmin) / (rmax - rmin)) * 100
    symbol_size = np.log(_normalized) * 10
    symbol_size = symbol_size + 1 if symbol_size > 0 else default_size
    return symbol_size

# 创建散点图对象
scatter = (
    Scatter()
    .add_xaxis(x_data)
    # .add_yaxis(
    #     "游戏数据", data.values.tolist(),
    #     label_opts=opts.LabelOpts(is_show=False),
    # )
    .set_global_opts(
        title_opts=opts.TitleOpts(title="游戏气泡图"),
        xaxis_opts=opts.AxisOpts(name=x, min_=0, type_="value"),
        yaxis_opts=opts.AxisOpts(name=y, min_=0, type_="value"),
        legend_opts=opts.LegendOpts(is_show=False),  # 显示图例
        tooltip_opts=opts.TooltipOpts(
            # 这里可以通过console.log(params)在控制台中打印出来
            formatter=JsCode(
                "function (params) { console.log(params); return params.name + '<br>' + params.data.release_date + '::release' + params.data.release_days + 'days<br>revenue: $' + params.data.revenue + '<br>dau: ' + params.data.dau ;}"
            )
        )
    )
)

for idx, game in data.iterrows():
    name, x_data, y_data = game['name'], game[x], game[y]
    y_data = min(y_data, y_max)
    symbol_size = cal_symbol_size(data, size, idx)
    symbol_size = max(3, symbol_size)
    game = game.to_dict()
    game['value'] = [x_data, y_data]
    g = (scatter
         .add_yaxis(name, [game],
                symbol_size= symbol_size,
                label_opts=opts.LabelOpts(is_show=False),
            )
        )



scatter.load_javascript()
# 渲染图表
scatter.render_notebook()
scatter.render("top100.html")

39.71787585701132
36.71775683559777
41.87607625006377
23.619796653703048
40.3317733393994
1
38.90782154822401
19.16793943043717
42.142306129568986
47.05170185988092
36.40756413501579
35.415419151238346
43.98647434103956
37.88330037823421
38.50199474161127
36.71343149382358
32.79918699128579
39.81833698972815
41.748298170968894
38.28879483135912
34.16991919961978
39.11737108714403
42.035416457747004
31.383199133658188
33.50895149278336
38.251091340002304
30.716326944113305
37.14918994463487
36.833556567982306
41.56529692167793
39.22904462389222
39.05757435924536
31.28782215510843
33.687887659576816
1
40.47394700020186
40.865865745206754
41.47709338616072
8.72610377447726
45.78702395061507
41.84354640107742
32.49599826053936
35.04750307907686
43.49016165223843
39.64837847250502
34.085185239942156
39.720369121354324
21.867420032139425
37.918039652569774
40.24461812957913
43.133365827690554
43.27458984118255
35.31150774471179
39.991721936117216
34.869375865578874
40.45183440266723
30.50851

C:\Users\HQ\AppData\Local\Temp\ipykernel_5180\2953979070.py:13: RuntimeWarning: divide by zero encountered in log
  symbol_size = np.log(_normalized) * 10


'D:\\SynologyDrive\\Sincetimes\\2024.12.24-20241217iosgametop100\\top100.html'